<a href="https://colab.research.google.com/github/nrajmalwar/EVA-6-Phase-1/blob/main/Session_04/Architectural_Basics/MNIST_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
!pip install torchsummary
from torchsummary import summary
from tqdm import tqdm

# Model Architecture

In [2]:
class Net(nn.Module):
    def __init__(self):

        super(Net, self).__init__()

        # First Block - Steadily Increase the number of channels to 24, followed by transition layer
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, 3), # 26, RF = 3
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(p=0.1),

            nn.Conv2d(16, 20, 3), # 24, RF = 5
            nn.ReLU(),
            nn.BatchNorm2d(20),
            nn.Dropout(p=0.1),

            nn.Conv2d(20, 24, 3), # 22, RF = 7
            nn.ReLU(),
            nn.BatchNorm2d(24),
            # No dropout before MaxPool

            # Transition Layer - MaxPool followed by 1x1 convolution
            nn.MaxPool2d(2, 2), # 11, RF = 14
            nn.Conv2d(24, 10, 1),
            nn.ReLU(),
            nn.BatchNorm2d(10),
        )
        
        # Second Block - Increase number of channels to 24, followed by 1x1 convolution (to be connected to FC layer)
        self.conv2 = nn.Sequential(
            nn.Conv2d(10, 16, 3), # 9, RF = 16
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.Dropout(p=0.1),

            nn.Conv2d(16, 20, 3), # 7, RF = 18
            nn.ReLU(),
            nn.BatchNorm2d(20),
            nn.Dropout(p=0.1),

            nn.Conv2d(20, 24, 3), # 5, RF = 20
            nn.ReLU(),
            nn.BatchNorm2d(24),
            nn.Dropout(p=0.1),

            nn.Conv2d(24, 16, 1), # 5, RF = 20  
            nn.ReLU(),
            nn.BatchNorm2d(16),
        )

        # Fully Connected Layer = No. of outputs
        self.output = nn.Sequential(
            nn.Linear(16, 10)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)

        # Global Average Pooling
        x = x.mean(dim=(2, 3))

        x = self.output(x)
        x = x.view(-1, 10)
        return F.log_softmax(input=x, dim=0)

# Model Summary

In [3]:
# use GPU if available
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Create an object of the network and transfer it to the device
model = Net().to(device)

# Print the model summary
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
           Dropout-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 20, 24, 24]           2,900
              ReLU-6           [-1, 20, 24, 24]               0
       BatchNorm2d-7           [-1, 20, 24, 24]              40
           Dropout-8           [-1, 20, 24, 24]               0
            Conv2d-9           [-1, 24, 22, 22]           4,344
             ReLU-10           [-1, 24, 22, 22]               0
      BatchNorm2d-11           [-1, 24, 22, 22]              48
        MaxPool2d-12           [-1, 24, 11, 11]               0
           Conv2d-13           [-1, 10, 11, 11]             250
             ReLU-14           [-1, 10,

# Data Preprocessing

In [4]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# Model Train and Test Functions

In [5]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        # optimizer.step()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# Model Training

In [6]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.003)

lambda1 = lambda epoch: 0.85 ** epoch
scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)

for epoch in range(20):
    print(f"\nEpoch number: {epoch+1}")
    train(model, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Epoch number: 1


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 2.6932, Accuracy: 9862/10000 (99%)


Epoch number: 2


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 2.6104, Accuracy: 9945/10000 (99%)


Epoch number: 19


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 2.6098, Accuracy: 9941/10000 (99%)


Epoch number: 20


loss=2.38149356842041 batch_id=468: 100%|██████████| 469/469 [00:13<00:00, 36.07it/s]



Test set: Average loss: 2.6136, Accuracy: 9941/10000 (99%)

